In [3]:
from sklearn.datasets import make_regression
import pandas as pd
import numpy as np 

In [16]:
X, y = make_regression(n_samples=200, n_features=4)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4'])
y = pd.Series(y)

In [18]:
X_train, X_test, y_train, y_test = X.iloc[:150], X.iloc[150:], y.iloc[:150], y.iloc[150:]

In [29]:
class MyKNNReg: 
    def __init__(self, k=3, metric='euclidean'): 
        self.k = k
        self.train_size = None
        self.metric = metric

        self.__valid()

    def __repr__(self): 
        return f'MyKNNReg class: k={self.k}'
   
    def __valid(self): 
        if self.metric not in ['euclidean', 'chebyshev', 'manhattan', 'cosine']:
            raise ValueError(f"Invalid metric '{self.metric}'. Choose from: 'euclidean', 'chebyshev', 'manhattan', 'cosine'.")
                               

    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.__X_train, self.__y_train = X.values, y.values 
        self.train_size = X.shape
        
    def __metric(self, X_train :pd.DataFrame, X_test: pd.DataFrame):
        '''Этот метод принимает тренеровачные и тестовые данные и на них считает метрики'''
        if self.metric == 'euclidean':
            dist = np.sqrt(np.sum((X_train-X_test)**2, axis=1))
            return dist 
        if self.metric == 'chebyshev': 
            dist = np.max(np.abs(X_train - X_test), axis=1)
            return dist 
        if self.metric == 'manhattan':
            dist = np.sum(np.abs(X_train - X_test), axis=1)
            return dist 
        if self.metric == 'cosine': 
            dist = 1 - (np.dot(X_train, X_test) /
            (np.linalg.norm(X_train, axis=1) * np.linalg.norm(X_test)))
            return dist

    def predict(self, X: pd.DataFrame): 
        X_test = X.values
        preds = []
        for x in X_test:                                               ## проходимся по каждому объекту из тестовой выборки  
            ## расстояние для всех объектов  train
            dist = self.__metric(self.__X_train, x)                     ## считаем расстояние по заданной метрике
            k_idx = np.argsort(dist)[:self.k]                          ## сортируем расстояние по возрасстанию (сортируются поиндексно) возращаем k элементов
            k_val = self.__y_train[k_idx]                              ## находим значения самый ближайших точек
            preds.append(np.mean(k_val))                               ## усредняем и длавляем в список preds
        return np.array(preds)

In [31]:
model = MyKNNReg()

In [33]:
model.fit(X_train, y_train)

In [35]:
model.predict(X_test)

array([ 1.35772229e+02,  7.64797327e+01,  7.10703319e+01,  2.77342092e+01,
        3.76875406e+01, -1.55854062e+02, -1.62322881e+02, -4.87181416e+01,
       -2.60714465e+01,  1.04699616e+02,  1.64213253e+02,  1.71410342e+02,
        2.21716448e+01, -9.61090226e+01, -1.32182156e+02,  7.64797327e+01,
        1.04648710e+02,  6.08532693e+01,  6.07831540e+01,  9.18642106e+01,
       -1.58091988e+02,  8.41077169e+01,  6.63290303e+00,  1.68461422e+02,
        6.18064457e+00, -9.44703055e+00, -2.00871998e+01,  3.60455264e+01,
        3.18140702e+01,  4.73888743e+01,  1.09566541e+02, -8.16449905e+01,
       -4.23343423e+01,  4.33200491e+01, -3.86841070e+01,  1.00595066e+02,
        8.41077169e+01,  2.60150708e+01,  7.64797327e+01,  4.33315009e+01,
        2.14642117e+02,  1.42340490e+02, -1.22918408e-01, -1.35823573e+02,
       -6.82708818e+01,  1.11890323e+01,  2.21717125e+02, -2.47989457e+01,
       -3.76930695e+01, -1.01080655e+02])